In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
import glob, re, os
import subprocess

from smt.applications.mixed_integer import (
    MixedIntegerContext,
    FLOAT,
    ENUM,
    ORD,
)

import scipy.linalg as lng

from smt.applications import EGO
from smt.surrogate_models import KRG
from smt.sampling_methods import LHS


import scienceplots
plt.style.use('default')
plt.style.use(['science', 'high-vis', 'grid'])

In [2]:
PATH_COST_CPP = "/work/utox/users/helleboid_work_utox/OptiDopi/build/apps/spad_cost_function"
RES_FILE = "RES/res_cost_func.csv"

os.makedirs("RES/", exist_ok=True)

def function_test_1d(length_intrinsic):
    doping_acceptor = 17.0
    subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{doping_acceptor}", RES_FILE])
    length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
    print(f)
    return total_cost

def function_test_2d(PARAMS):
    #print(PARAMS)
    Y = []
    for idx in range(len(PARAMS)):
        length_intrinsic = PARAMS[idx,0]
        #print(f'{length_intrinsic=}')
        log_doping_acceptor =  PARAMS[idx,1]
        subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{log_doping_acceptor}", RES_FILE])
        length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
        Y.append(total_cost)
        print(f"Total cost: {total_cost}")
    return np.array(Y)

vfunc = np.vectorize(function_test_1d)

vfunc2d = np.vectorize(function_test_2d)


In [4]:
min_length_intrinsic = 0.0
max_length_intrinsic = 1.0
min_log_acceptor = 16.5
max_log_acceptor = 19.0


n_iter = 10000
xtypes = [FLOAT, FLOAT]
#mixint = MixedIntegerContext(xtypes, xlimits)
xlimits = np.array([[min_length_intrinsic, max_length_intrinsic], [min_log_acceptor, max_log_acceptor]])

qEI = "KB"
sm = KRG(print_global=False)
mixint = MixedIntegerContext(xtypes, xlimits)
n_doe = 5
sampling = LHS(xlimits=xlimits)
xdoe = sampling(n_doe)
ydoe = function_test_2d(xdoe)



Total cost: 135.5281
Total cost: 209.2162
Total cost: 1214.816
Total cost: 580.3904
Total cost: 837.0175


In [5]:
criterion = "SBO"  #'EI' or 'SBO' or 'LCB'
ego = EGO(n_iter=n_iter, criterion=criterion, xdoe=xdoe, xlimits=xlimits, xtypes=xtypes, qEI=qEI,
          enable_tunneling=True)
x_opt, y_opt, _, x_data, y_data = ego.optimize(fun=function_test_2d)

Total cost: 135.5281
Total cost: 209.2159
Total cost: 1214.816
Total cost: 580.3904
Total cost: 837.0174
Total cost: 169.1432
Total cost: -34.83819
Total cost: 5217.243
Total cost: 222.0632
Total cost: 102.4776
Total cost: -32.70046
Total cost: -39.60162
Total cost: -39.59703
Total cost: -39.59853
Total cost: 492.5773
Total cost: 4.571124
Total cost: -30.83186
Total cost: 229.1192
Total cost: 824.7776
Total cost: -38.81456
Total cost: -6.841653
Total cost: -41.28964
Total cost: -41.83048
Total cost: -31.72175
Total cost: 71.2932
Total cost: -4.800855
Total cost: -50.90315
Total cost: 25.95115
Total cost: 39.31174
Total cost: 171.9098
Total cost: 190.6726
Total cost: -25.46549
Total cost: 65.94753
Total cost: -51.63542
Total cost: -31.00346
Total cost: -31.36374
Total cost: -61.50692
Total cost: 102.5027
Total cost: 10.68881
Total cost: 210.3924
Total cost: -12.92183
Total cost: -41.64768
Total cost: -60.98754
Total cost: -48.13328
Total cost: -19.78864
Total cost: -13.85188
Total cost:

FileNotFoundError: [Errno 2] No such file or directory: '/work/utox/users/helleboid_work_utox/OptiDopi/build/apps/spad_cost_function'

In [ ]:
print("Minimum in x={} with f(x)={:.1f}".format(x_opt, float(y_opt)))
print("Minimum in typed x={}".format(ego.mixint.cast_to_mixed_integer(x_opt)))

min_ref = -15
mini = np.zeros(n_iter)
for k in range(n_iter):
    mini[k] = np.log(np.abs(np.min(y_data[0 : k + n_doe - 1]) - min_ref))
x_plot = np.linspace(1, n_iter + 0.5, n_iter)
u = max(np.floor(max(mini)) + 1, -100)
l = max(np.floor(min(mini)) - 0.2, -10)
fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
axes.plot(x_plot, mini, color="r")
axes.set_ylim([l, u])
plt.title("minimum convergence plot", loc="center")
plt.xlabel("number of iterations")
plt.ylabel("log of the difference w.r.t the best")
plt.show()